# Fetching the dataset


In [1]:
# dependencies
import os
import pandas as pd
import numpy as np

In [ ]:
# fetching the dataset

data = pd.read_parquet(
    "hf://datasets/michsethowusu/twi-sentiments-corpus/data/train-00000-of-00001-871f629b5e597578.parquet"
)

print(f"len data1: {len(data)}")

len data1: 432647


In [23]:
data.head()

,Twi,sentiment,__index_level_0__
0,Nyansa mu na woyɛɛ ne nyinaa;,Positive,0
1,"alo yɛngɛ sone,",Negative,1
2,Wosɛe wɔn a wɔnni wo nokorɛ nyinaa.,Negative,2
3,Akatua bɛn na ɔde bɛma wɔn a wɔde gyidi som no?,Positive,3
4,mepɛ Onyankopɔn ho nimdeɛ sen ɔhyew afɔre.,Positive,4


In [24]:
data = data["Twi"]

In [25]:
data.head()

0                      Nyansa mu na woyɛɛ ne nyinaa;
1                                    alo yɛngɛ sone,
2                Wosɛe wɔn a wɔnni wo nokorɛ nyinaa.
3    Akatua bɛn na ɔde bɛma wɔn a wɔde gyidi som no?
4         mepɛ Onyankopɔn ho nimdeɛ sen ɔhyew afɔre.
Name: Twi, dtype: object

In [ ]:
data.to_csv("./data/processed/twiSentences.csv")

In [27]:
twi = pd.read_csv("./data/processed/twiSentences.csv")

# Preprocessing


In [8]:
import re
import nltk
import pickle
import string
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\os\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def cleanText(text):
    text = text.lower()